# Error Analysis 
Did tracts falsely predicted as "gentrifying" for a given year, go on to gentrify in the future?

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import geopandas as gpd

import sys
sys.path.insert(0, './')
from functions import *
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet("Data/Cleaned/after_kriging.parquet")
features = pd.read_parquet("Data/Cleaned/post_feat_engineering.parquet")
labels = pd.read_parquet("Data/Cleaned/labels_may5.parquet")
future_preds = pd.read_parquet("Data/Cleaned/future_preds.parquet")

preds20 = pd.read_parquet("Data/Cleaned/2020_preds.parquet")
preds21 = pd.read_parquet("Data/Cleaned/2021_preds.parquet")
preds22 = pd.read_parquet("Data/Cleaned/2022_preds.parquet")
preds23 = pd.read_parquet("Data/Cleaned/2023_preds.parquet")


## For 2020 Preds

In [3]:
# join all the yearly preds together
preds20 = preds20[['GEOID','label','preds']].rename(columns={'preds':'preds2020', 'label':'label2020'})
preds = preds20.merge(preds21[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2021'})
preds = preds.merge(preds22[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2022'})
preds = preds.merge(preds23[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2023'})
preds.drop(preds[preds.label2020==0].index, inplace=True) # drop non-eligible tracts

# label false pos and true neg in new column
preds['TN'] = np.where((preds.label2020==1)&(preds.preds2020==1),1,0)
preds['FP'] = np.where((preds.label2020==1)&(preds.preds2020==2),1,0)

In [4]:
# new column to flag tracts that gentrified ANYTIME AFTER 2020
preds['gentrified_later'] = np.where(((preds['labels2021'].isin([0,2]))|
                                     (preds['labels2022'].isin([0,2]))|
                                     (preds['labels2023'].isin([0,2]))),1,0)


In [256]:
# how many FP's ended up gentrifying in 2021?
fp["labels2021"].value_counts()


labels2021
1    60
2    14
0     9
Name: count, dtype: int64

In [309]:
# how many TN's ended up gentrifying in 2021?
tn["labels2021"].value_counts()

labels2021
1    304
0     33
2     20
Name: count, dtype: int64

In [310]:
# how many FP's ended up gentrifying in 2021 or 2022?
preds['gentrified_21_22'] = np.where(((preds['labels2021'].isin([0,2]))|
                                      ((preds['labels2022'].isin([0,2])))),1,0)
fp = preds[(preds.FP==1)]
fp["gentrified_21_22"].value_counts()

gentrified_21_22
0    50
1    33
Name: count, dtype: int64

In [311]:
# how many TN's ended up gentrifying in 2021 or 2022?
tn = preds[(preds.TN==1)]
tn["gentrified_21_22"].value_counts()

gentrified_21_22
0    281
1     76
Name: count, dtype: int64

In [32]:
# how many FP's ended up gentrifying later?
fp = preds[(preds.FP==1)]
fp["gentrified_later"].value_counts()


gentrified_later
0    44
1    39
Name: count, dtype: int64

In [33]:
# how many TN's ended up gentrifying later?
tn = preds[(preds.TN==1)]
tn["gentrified_later"].value_counts()

gentrified_later
0    257
1    100
Name: count, dtype: int64

### Compare FP's to TN's

In [34]:
# Compare TN's to FP's
fp_ids = fp[fp.gentrified_later==1]['GEOID']
tn_ids = tn[tn.gentrified_later==0]['GEOID']
preds20 = pd.read_parquet("Data/Cleaned/2020_preds.parquet")

fp = preds20[preds20.GEOID.isin(fp_ids)]
tn = preds20[preds20.GEOID.isin(tn_ids)]

# look at feature values for each of these two groups
fp_mean = fp.drop(['GEOID','pred_year'],axis=1).mean()
fp_mean.name='fp'
tn_mean = tn.drop(['GEOID','pred_year'],axis=1).mean()
tn_mean.name='tn'

diffs = pd.concat([fp_mean,tn_mean], axis=1)
# diffs.to_csv("Data/Cleaned/TN_vs_FP.csv")
diffs


,fp,tn
total_pop,0.009811,0.013410
percent_hh_income_assist,0.065260,0.075437
percent_hh_snap,0.053659,0.053860
percent_hh_english,0.020028,0.004338
percent_work_agriculture,-0.003441,-0.003461
percent_work_construction,0.058394,0.111250
percent_work_retail,0.016648,0.073598
percent_work_finance,0.058620,0.060854
percent_work_stem,0.151619,0.068123
percent_work_edu_health,0.013608,0.013082


## For 2021 Preds

In [321]:
# join all the yearly preds together
preds21 = preds21[['GEOID','label','preds']].rename(columns={'preds':'preds2021', 'label':'label2021'})
preds = preds21.merge(preds22[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2022'})
preds = preds.merge(preds23[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2023'})
preds.drop(preds[preds.label2021==0].index, inplace=True) # drop non-eligible tracts

# label false pos and false neg in new column
preds['TN'] = np.where((preds.label2021==1)&(preds.preds2021==1),1,0)
preds['FP'] = np.where((preds.label2021==1)&(preds.preds2021==2),1,0)

In [322]:
# new column to flag tracts that gentrified ANYTIME AFTER 2021
preds['gentrified_later'] = np.where(((preds['labels2022'].isin([0,2]))|
                                     (preds['labels2023'].isin([0,2]))),1,0)

In [323]:
# how many FP's ended up gentrifying later?
fp = preds[(preds.FP==1)]
fp["gentrified_later"].value_counts()

gentrified_later
1    17
0    16
Name: count, dtype: int64

In [324]:
# how many TN's ended up gentrifying later?
tn = preds[(preds.TN==1)]
tn["gentrified_later"].value_counts()

gentrified_later
0    337
1     89
Name: count, dtype: int64

In [325]:
# how many FP's ended up gentrifying in 2022?
fp["labels2022"].value_counts()

labels2022
1    17
2    13
0     3
Name: count, dtype: int64

In [326]:
# how many TN's ended up gentrifying in 2022?
tn["labels2022"].value_counts()

labels2022
1    378
0     28
2     20
Name: count, dtype: int64

### For 2022 Preds

In [327]:
# join all the yearly preds together
preds22 = preds22[['GEOID','label','preds']].rename(columns={'preds':'preds2022', 'label':'label2022'})
preds = preds22.merge(preds23[['label', 'GEOID']], on='GEOID').rename(columns={'label':'labels2023'})
preds.drop(preds[preds.label2022==0].index, inplace=True) # drop non-eligible tracts

# label false pos and false neg in new column
preds['TN'] = np.where((preds.label2022==1)&(preds.preds2022==1),1,0)
preds['FP'] = np.where((preds.label2022==1)&(preds.preds2022==2),1,0)

In [328]:
# how many FP's ended up gentrifying in 2023?
fp = preds[(preds.FP==1)]
fp["labels2023"].value_counts()

labels2023
1    36
2    12
0     1
Name: count, dtype: int64

In [329]:
# how many TN's ended up gentrifying in 2023?
tn = preds[(preds.TN==1)]
tn["labels2023"].value_counts()

labels2023
1    373
0     33
2     20
Name: count, dtype: int64